## Point Classifier
Determine wheter a point is above or below an arbitrary line in 2D space<br>
Last update: 12/23/23

In [1]:
import cupy as np
from importlib import reload
from sandbox import model, layers, activations, costs, utils

In [2]:
line = lambda x: x**2 - 2
mean = 0
sd = 5
num_train = 100
num_test = 100

# Create training and test input data - Points in 2D space with given mean and sd
train_x = np.random.normal(mean, sd, (num_train, 2))
test_x = np.random.normal(mean, sd, (num_test, 2))

# Create labels by determining whether the input is above or below the line
train_y = np.where(train_x[:, 1] > line(train_x[:, 0]), 1, 0).reshape(-1, 1)
test_y = np.where(test_x[:, 1] > line(test_x[:, 0]), 1, 0).reshape(-1, 1)

In [4]:
reload(model)

# Create model
point = model.Model(cuda=True)
point.add(layers.Dense(units=8, activation=activations.ReLU()))
point.add(layers.Dense(units=8, activation=activations.ReLU()))
point.add(layers.Dense(units=1, activation=activations.Sigmoid()))
point.configure(cost_type=costs.BinaryCrossentropy())

# Train model
point.train(train_x, train_y, learning_rate=0.01, epochs=300, verbose=True)

Cost on epoch 0: 0.28285
Cost on epoch 30: 0.23497
Cost on epoch 60: 0.21216
Cost on epoch 90: 0.19772
Cost on epoch 120: 0.18696
Cost on epoch 150: 0.17842
Cost on epoch 180: 0.17122
Cost on epoch 210: 0.16501
Cost on epoch 240: 0.15932
Cost on epoch 270: 0.15468
Cost on epoch 300: 0.15007


In [6]:
# Evaluate model on test data
pred = utils.binary_round(point.predict(test_x))
print('Testing Accuracy: ' + str(np.round(np.sum((pred == test_y)/test_x.shape[0]), decimals=5)))

Testing Accuracy: 0.89
